## Módulo Multijugador


Hasta el momento hemos podido comparar estrategias entre ellas, valorando cuales obtienen mejores resultados en partidas unijugador. Podría ser intuitivo pensar que las buenas estrategias del modo unijugador seguirán siendo buenas en el modo multijugador pero ¿es esto cierto en el Snake? <br>
... <br>
Para responder a esta pregunta hemos reprogramado el archivo **juego_base.py** para que pueda soportar varios agentes. El nuevo archivo tiene como nombre **juego_base_multijugador.py** (muy intuitivo). Los principales cambios realizados son los que siguen:
<br>
<img src= "media\imagenes\FormatoSnake.png" width= 370 style="float: right;">

1. La información de las serpientes, que antes era una lista de tuplas con coordenadas, ahora es una lista de listas de tuplas, donde cada lista contiene una serpiente diferente. *(ver figura a la derecha)* ------->

2. Creamos una lista con todos los agentes, que podemos iterarar para obtener la acciones que realizan en cada ronda.

3. Los agentes necesitan tener algo que los identifique, por eso se añaden **números ID**. Al invocar un agente, debes meter un número ID único que lo distinga del resto. De esta forma, podemos realizar eliminaciones de agentes por ID o mostrar el ganador al final de una partida.

4. El método upload(action) pasa a recibir una lista de acciones en vez de una única acción, y actúa cambiando el estado del tablero cuando las serpientes realizan esas acciones.

5. La nueva variable **self.bodies** contiene las coordenadas de TODOS los cuerpos. Así, es fácil comprender cuando una <br> serpiente choca con una porción de cuerpo, dando igual a quién le pertenezca el cacho.

6. Las posiciones de comienzo de los agentes varían con respecto al número de agentes usados. Hasta 4 agentes, se rellenan las esquinas del tablero. De 4 a 8 agentes se añaden puntos de comienzo en las mitades de los lados.

7. La condición de victoria ahora es aguantar hasta que el resto de serpientes mueran y ser el último en el tablero. Esto se da cuando la longitud de la lista de snakes es 1, que se traduce en un solo jugador. Cuando esto ocurre, el juego acaba y se devuelve el ID de la serpiente ganadora.

8. Como detalle estético, usamos los IDs multplicados por ciertas cantidades para establecer el código RGB de los agentes. Puesto que cada agente tiene un ID único, el color de su cuerpo también lo será y se mostrará diferente al resto durante la partida.

*NOTA: Cuando dos serpientes choquen sus cabezas, la más grande se comerá a la pequeña.*

Con todos estos cambios hechos, obtenemos un juego base capaz de gestionar los agentes. Ahora, toca ajustar los agentes para que tengan en cuenta el resto de cuerpos aparte del propio, y sepan actuar al respecto. <br>
Los cambios que se realizan en la heurística de los agentes son triviales, pues solo hay que usar la lista de bodies para entender las posiciones del resto de jugadores, y los pesos se distribuyen de forma consecuente al tipo de heurística.

### Avoider Multijugador
Cambiando el modelo **avoider** obtenemos el increíble **AVOIDER MULTIJUGADOR** (sí, en mayúsculas). Actúa igual que el avoider, pero adaptado para poder interactuar con otros agentes. <br>
Ejecuta la siguiente celda para probar la **primera partida multijugador**.

In [10]:
import dependencias

# Definimos nuestros agentes
Borja = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(4321)
Benjamin = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(1234)

# Creamos el juego
game = dependencias.Snake_game_MultiPlayer((25, 25), 40, [Borja, Benjamin])  # Ponemos mucha comida, para que se la encuentren sin querer

# Ejecutamos
game.play_with_pygame()

El ganador es: Snake1234


No hay gran cambio en como actúan con respecto a partidas de un jugador, puesto que no tienen muchas cosas en cuenta.

### Chaser Multijugador
Con un par de cambios en el modelo **chaser** obtenemos nuestro primer **CHASER MULTIJUGADOR**. Veamos una partida entre ellos.

In [11]:
import dependencias

# Definimos nuestros agentes
Ortega = dependencias.Agentes_multiplayer.ChaserAgentMp(231)
Gasset = dependencias.Agentes_multiplayer.ChaserAgentMp(728)

# Creamos el juego
game = dependencias.Snake_game_MultiPlayer((25, 25), 5, [Ortega, Gasset])

# Ejecutamos
game.play_with_pygame()  # Es normal que las partidas acaben casi instantáneamente, pues estos agentes no saben esquivar la muerte.

El ganador es: Snake728


Efectivamente, los chaser siguen siendo "malos" pues no evitan la muerte.

## Combinación de Agentes

### Chowder Multijugador

Como se pudo ver en el módulo de agentes de un solo jugador, las mejores estrategias surgen de mezclar otras más pequeñas. Implementemos eso en multijugador. La mecánica será la misma, mediante unos pesos se hacen sumas ponderadas de las heurísticas y se elige la opción más puntuada. Probemos a mergear un AVOIDER MULTIPLAYER con un CHASER MULTIPLAYER, creando un **CHOWDER MULTIPLAYER**. <br>
Ahora vamos a enfrentar dos de estos modelos usando distintos pesos. <br> <br>
*Nota: Como se aprecia en el código, tanto el agente combinado como sus subagentes deben usar el mismo ID para que el código funcione y se asocien entre ellos*

In [12]:
import dependencias

Ortega1 = dependencias.Agentes_multiplayer.ChaserAgentMp(33333)
Borja1 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(33333)
Ortega2 = dependencias.Agentes_multiplayer.ChaserAgentMp(44444)
Borja2 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(44444)

chowder = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega1, Borja1], weights = (0.5, 1), id = 33333)  # Combinamos con pesos
chowdar = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega2, Borja2], weights = (1, 0.5), id = 44444)


game = dependencias.Snake_game_MultiPlayer((25, 25), 5, [chowder, chowdar])

game.play_with_pygame()

El ganador es: Snake33333


Se empiezan a ver resultados. Estos agentes ya empiezan a ser relativamente buenos, evitando la muerte y persiguiendo la comida. Solo con esto, ya podemos enfrentar muchos agentes combinados con distintos pesos para ver las mezclas que más funcionan. Aprovechemos el momento también para ampliar el número de jugadores. Veamos una partida entre 5 jugadores.

In [7]:
import dependencias

Ortega1 = dependencias.Agentes_multiplayer.ChaserAgentMp(11)
Borja1 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(11)
Ortega2 = dependencias.Agentes_multiplayer.ChaserAgentMp(22)
Borja2 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(22)
Ortega3 = dependencias.Agentes_multiplayer.ChaserAgentMp(33)
Borja3 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(33)
Ortega4 = dependencias.Agentes_multiplayer.ChaserAgentMp(44)
Borja4 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(44)
Ortega5 = dependencias.Agentes_multiplayer.ChaserAgentMp(55)
Borja5 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(55)

chowder1 = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega1, Borja1], weights = (0.5, 1), id = 11)
chowder2 = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega2, Borja2], weights = (0.5, 1.2), id = 22)
chowder3 = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega3, Borja3], weights = (0.3, 1.1), id = 33)
chowder4 = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega4, Borja4], weights = (0.4, 0.9), id = 44)
chowder5 = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega5, Borja5], weights = (0.6, 1.5), id = 55)

game = dependencias.Snake_game_MultiPlayer((40, 40), 15, [chowder1, chowder2, chowder3, chowder4, chowder5])

game.play_with_pygame()

El ganador es: Snake55


### Deep-Chowder y Tail-Chasser


Mezclando el Chowder con un modelo de búsqueda profunda obtenemos Deep-Chowder. Vamos a probarlo enfrentandolo contra un chowder normal.

In [14]:
import dependencias

Ortega1 = dependencias.Agentes_multiplayer.ChaserAgentMp(11)
Borja1 = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(11)
chowder1 = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [Ortega1, Borja1], weights = (0.5, 1), id = 11)

avoider = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(69)
deeper = dependencias.Agentes_multiplayer.Busqueda_anchura_Mp(69)
chaser = dependencias.Agentes_multiplayer.ChaserAgentMp(69)

deep_chowder = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [avoider, deeper, chaser], weights = (1, 2, 0.5), id = 69)

game = dependencias.Snake_game_MultiPlayer((25, 25), 5, [chowder1, deep_chowder])

game.play_with_pygame()  # Suele ganar el modelo Deep_Chowder (Snake69)

El ganador es: Snake69


El modelo Deep_Chowder sale de manera satisfactoria de las situaciones complicadas que le genera la otra serpiente. Salvo en situaciones contadas de mala suerte, Deep_Chowder se libra de encerronas de Chowder, mientras que a la inversa no ocurre.
<br><br>
Entre tantas pruebas se va visualizando una conclusión final. Efectivamente, los modelos buenos unijugador acaban siendo buenos también contra otros jugadores. Cuánto más trabajamos en los parámetros de un modelo, los ajustamos, y usamos distintos sub-agentes, mejor acaba siendo el modelo al enfrentarlo contra otros jugadores. <br>
<br>
Implementamos el Tail_Chasser y lo probamos añadiéndolo a Deep_Chowder. 

In [16]:
import dependencias

tail_chasser = dependencias.Agentes_multiplayer.Tail_Chasser_Mp(99)
avoiderh = dependencias.Agentes_multiplayer.Avoid_inmediate_death_Mp(99)
deeperh = dependencias.Agentes_multiplayer.Busqueda_anchura_Mp(99)
chaserh = dependencias.Agentes_multiplayer.ChaserAgentMp(99)

deep_chowder_tail = dependencias.Agentes_multiplayer.Combined_agent_Mp(agentes = [avoiderh, deeperh, chaserh, tail_chasser], weights = (1, 2, 0.5, 3), id = 99)

game = dependencias.Snake_game_MultiPlayer((25, 25), 5, [chowder1, chowder2, deep_chowder, deep_chowder_tail])  # Lo enfrentamos contra chowders y deepchowders 

game.play_with_pygame()

El ganador es: Snake22


En el caso del Tail_Chasser, podemos observar que pese a ser muy bueno en unijugador no aporta mucho en multijugador pues las situaciones con poco espacio (donde es bueno este modelo) casi no se producen.

## Reflexión Final

Hemos comprobado la hipótesis propuesta al inicio del módulo. Sin embargo, en el proceso de construir este trabajo, descubrimos que el juego Snake encierra una profundidad insospechada. Lo que parecía un simple ejercicio técnico nos abrió las puertas a un vasto campo de estudio: desde las relaciones entre distintos modelos de serpientes hasta la influencia decisiva de las características del tablero en el rendimiento de estas.

Snake, un juego que inicialmente concebimos como un tablero de píxeles con reglas sencillas, resultó ser un microcosmos de posibilidades. ¿No es fascinante cómo un sistema tan pequeño puede encapsular principios aplicables a problemas más grandes, incluso personales? Quizás, en cada giro y movimiento de la serpiente, se esconden patrones que resuenan con la forma en que enfrentamos desafíos, tomamos decisiones y navegamos por la incertidumbre de la vida misma. Quizás, Eva solo intentaba seguir los pasos de un ente omnipotente con conocimientos ancestrales.

Al final, tal vez Snake nunca fue solo un juego. Tal vez sea una metáfora: una lección sobre cómo incluso los sistemas más aparentemente simples contienen complejidades que nos desafían a mirar más allá de lo evidente, a ser pacientes, estratégicos, y, sobre todo, humildes ante lo que creíamos entender por completo. Porque, al igual que la serpiente, nosotros también seguimos creciendo, aprendiendo y buscando el equilibrio en cada movimiento.
<br>

<center>
<b>Un trabajo por:</b> <br><br>
Álvaro Ramiro <br>

Jordi Hamberg <br>

Héctor Sancho <br><br>

 <br><br>
<img src= "media\imagenes\Logo.jpg" width = 300>

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<i>Larga vida a Senollop</i>
</center>

